In [2]:
!pip install win32com
! pip install pythoncom

ERROR: Could not find a version that satisfies the requirement win32com
ERROR: No matching distribution found for win32com
ERROR: Could not find a version that satisfies the requirement pythoncom
ERROR: No matching distribution found for pythoncom


In [1]:
#-*- coding: utf-8 -*-

import win32com.client
import pythoncom
import pandas as pd
import time

#세션 클래스
class XASessionEvents:
    login_state = 0
    def OnLogin(self, code, msg):
        if code == "0000":
            print(msg)
            XASessionEvents.login_state = 1
        else:
            print("로그인 실패 : ",msg)

class XAQueryEvent:
    query_state = 0
    isWaiting = False
    isRequested = False

    def OnReceiveData(self, code):
        XAQueryEvent.query_state = 1
        isRequested = False
        isWaiting = False

    def OnReceiveMessage(self, bIsSystemError, szMessageCode, szMessage):
        # TR 전송 제한 처리
        #print("code:", szMessage)
        if szMessage == 'TR의 시간당 전송제한에 걸렸습니다.':
            print(szMessageCode, szMessage)
            forced_waiting = True
            time.sleep(1)
            
class XAQueryEventsT1102:  # 단일 주식종목 조회
    query_state = 0
    def OnReceiveData(self, code):
        XAQueryEventsT1102.query_state = 1



class XAQueryEventsT8413:
    query_state = 0
    def OnReceiveData(self, code):
        XAQueryEventsT8413.query_state = 1
        
    def OnReceiveMessage(self, bIsSystemError, szMessageCode, szMessage):
        # TR 전송 제한 처리
        #print("code:", szMessage)
        if szMessage == 'TR의 시간당 전송제한에 걸렸습니다.':
            print(szMessageCode, szMessage)
            #XAQueryEvent.forced_waiting = True
            #time.sleep(1)

class XAQueryEventsT8419:   # 코스피, 코스닥 등 지수 조회
    query_state = 0
    def OnReceiveData(self, code):
        XAQueryEventsT8419.query_state = 1  
        
    def OnReceiveMessage(self, bIsSystemError, szMessageCode, szMessage):
        # TR 전송 제한 처리
        #print("code:", szMessage)
        if szMessage == 'TR의 시간당 전송제한에 걸렸습니다.':
            print(szMessageCode, szMessage)
            #XAQueryEvent.forced_waiting = True
            #time.sleep(1)
        
class XAQueryEventsT8424:   # 업종명, 코드 조회
    query_state = 0
    def OnReceiveData(self, code):
        XAQueryEventsT8424.query_state = 1            
            
class XAQueryEventsT8430:   #코스피, 코스닥 전체 주식종목 조회
    query_state = 0
    def OnReceiveData(self, code):
        XAQueryEventsT8430.query_state = 1
        
        
        
def Login(mode):
    id = "ellun77"
    passwd = "ataka900"
    certificate_passwd = "ataka0092021(_"
    instXASession = win32com.client.DispatchWithEvents("XA_Session.XASession", XASessionEvents)
    if(mode == "real"):
        instXASession.ConnectServer("hts.ebestsec.co.kr", 20001)  # 실제투자 서버는 hts.ebestsec.co.kr
    else:
        instXASession.ConnectServer("demo.ebestsec.co.kr", 20001)  # 실제투자 서버는 hts.ebestsec.co.kr
    instXASession.Login(id, passwd, certificate_passwd, 0, 0)

    # 로그인 성공 이벤트를 받을 때 까지 대기함
    while XASessionEvents.login_state == 0:
        pythoncom.PumpWaitingMessages()
        
def Disconnect():
    if XASessionEvents.login_state == 1:
        instXASession = win32com.client.DispatchWithEvents("XA_Session.XASession", XASessionEvents)
        instXASession.DisconnectServer()
        XASessionEvents.login_state = 0
        XASessionEvents.connect_state = 0
        print("연결을 종료합니다.")
    else:
        print('현재 연결이 되어있지 않습니다.')

In [2]:
# 서버 접속 및 로그인
instXASession = win32com.client.DispatchWithEvents("XA_Session.XASession", XASessionEvents)
    
if instXASession.IsConnected()  == False:
    Login('demo')
    print('서버에 연결되었습니다.')
else:
    print('이미 서버에 연결 되어있습니다.')    

# 로그인 성공 이벤트를 받을 때 까지 대기함
while XASessionEvents.login_state == 0:
    pythoncom.PumpWaitingMessages()

XASessionEvents.login_state = 1
XASessionEvents.connect_state = 1 


# 계좌 가져오기
nAccount = instXASession.GetAccountListCount()
for i in range(nAccount):
    account = instXASession.GetAccountList(i)
    print(account)

로그인 성공
서버에 연결되었습니다.
55501065006
55551040666


In [4]:
Disconnect()


연결을 종료합니다.


In [3]:
instXAQueryT4201 = win32com.client.DispatchWithEvents("XA_DataSet.XAQuery", XAQueryEvent)
instXAQueryT4201.ResFileName = "C:/eBEST/xingAPI/Res/t4201.res"
instXAQueryT4201.SetFieldData("t4201InBlock", "gubun", 0, "2")  # 0:틱, 1:분, 2:일, 3:주, 4:월
instXAQueryT4201.SetFieldData("t4201InBlock", "sdate", 0, "20140101") #시작일 (최대 500일까지만 나옴)
instXAQueryT4201.SetFieldData("t4201InBlock", "edate", 0, "20160629")  #마지막일 (2014-01-01,2016-04-22)
print(instXAQueryT4201.GetTRCountLimit('t4201'))
print(instXAQueryT4201.GetTRCountRequest('t4201'))
print(instXAQueryT4201.GetTRCountBaseSec('t4201'))

200
-1
1


In [3]:
# 전 종목 주가 데이터 가져오기

j = 0
tr_frequency = 1.1

column_dailychart = ['shcode', 'gubun', 'date', 'open', 'high', 'low', 'close','jdiff_vol', 'value', 'jongchk', 'rate', 'pricechk', 'ratevalue', 'sign']
row = list(range(len(column_dailychart)))
rows = list()

print("종목정보 가져오는 중...")
stockitems = pd.read_csv('stockitems.csv')
stockitems['type'] = stockitems['gubun']
stockitems.loc[stockitems['etfgubun'] == 1, 'type'] = 3  #ETF인 경우 type=3 으로 지정
stockitems.loc[stockitems['etfgubun'] == 2, 'type'] = 4  #ETN인 경우 type=4 로 지정

print('종목정보 ', len(stockitems), '개 가져오기 완료')

#instXAQueryEvent = XAQueryEvent()

# MySQL Connection 연결
instXAQueryT8413 = win32com.client.DispatchWithEvents("XA_DataSet.XAQuery", XAQueryEventsT8413)
instXAQueryT8413.ResFileName = "C:/eBEST/xingAPI/Res/t8413.res"
instXAQueryT8413.SetFieldData("t8413InBlock", "gubun", 0, "2")  # 2:일, 3:주, 4:월
instXAQueryT8413.SetFieldData("t8413InBlock", "sdate", 0, "20140101") #시작일 (비압축은 최대 500일, 압축은 최대 2000일 까지 나옴)
instXAQueryT8413.SetFieldData("t8413InBlock", "edate", 0, "20210330")  #마지막일
instXAQueryT8413.SetFieldData("t8413InBlock", "comp_yn", 0, 'Y')
isCompressed = True

#query_limit =  instXAQueryT8413.GetTRCountLimit('t8413')
query_limit =  200
query_limit_flag = True
dailychart = pd.DataFrame(data = rows, columns= column_dailychart)

dailychart_column_type = {'shcode':str, 'gubun':int, 'date':str, 'open':int, 'high':int, 'low':int, 'close':int,'jdiff_vol':int, 'value':int, 'jongchk':int,
               'rate':float, 'pricechk':int, 'ratevalue':int, 'sign':int}

dailychart_original = pd.read_csv('dailychart3.csv', dtype=dailychart_column_type)
n_items = len(dailychart_original['shcode'].unique())
print(n_items)

for idx, stockitem in stockitems.iterrows():
    if j < n_items:
        j = j + 1
        continue
    
    if j == n_items+200:
        break
   # if stockitem['shcode']not in ['900260', '900310', '950140', '950180', '950220']:
   #     j = j + 1
     #   continue
        
    if (instXAQueryT8413.GetTRCountRequest('t8413') >= query_limit) and query_limit_flag:
        print("10분내 호출 제한 수에 걸렸습니다. 5초 단위로 호출을 시작합니다." )
        tr_frequency = 5.1
        query_limit_flag = False
        break

    time.sleep(tr_frequency)
    print(j, ' / 코드 : ', stockitem['shcode'], ' / 종목명 : ' , stockitem['hname'])
    
    instXAQueryT8413.SetFieldData("t8413InBlock", "shcode", 0, stockitem['shcode'])
    instXAQueryT8413.Request(0)
    
    while XAQueryEventsT8413.query_state == 0:
        pythoncom.PumpWaitingMessages()
        #print('waiting')

     
    XAQueryEventsT8413.query_state = 0  # 쿼리가 완료되면 다시 상태를 되돌려 놓음
    shcode2 =  instXAQueryT8413.GetFieldData("t8413OutBlock","shcode",0) # 종목코드
    #print(shcode2)
    #g_count =  instXAQueryT8413.GetBlockCount("t8413OutBlock1")
    if isCompressed:
        nDecompSize = instXAQueryT8413.Decompress("t8413OutBlock1")
        pythoncom.PumpWaitingMessages()
        if nDecompSize > 0:
            g_count = instXAQueryT8413.GetBlockCount("t8413OutBlock1")
        else:
            g_count = 0
    else:
        g_count =  instXAQueryT8413.GetBlockCount("t8413OutBlock1")
    
    """    
    while True:
        while XAQueryEvent.query_responded == 0:
            if not XAQueryEvent.isRequested:
                instXAQueryT8413.Request(0)
                print('waiting.request',   XAQueryEvent.query_responded, XAQueryEvent.isRequested)
                XAQueryEvent.isRequested = True
            time.sleep(0.5)
            print('waiting...',   XAQueryEvent.query_responded, XAQueryEvent.isRequested)
            pythoncom.PumpWaitingMessages()

        XAQueryEvent.query_responded = 0
        shcode2 = instXAQueryT8413.GetFieldData("t8413OutBlock","shcode",0)

        if(stockitem['shcode'] != shcode2):
            print("종목 일치하지 않음, 대기..")
            time.sleep(1)
            instXAQueryT8413.Request(0)
        else:
            break
    """
    
    for i in range(g_count):
        row[0] =  shcode2
        row[1] = stockitem['type']  # 코스피, 코스닥 여부
        row[2] = instXAQueryT8413.GetFieldData("t8413OutBlock1", "date", i)  # 날짜
        row[3] = instXAQueryT8413.GetFieldData("t8413OutBlock1", "open", i)  # 시가
        row[4] = instXAQueryT8413.GetFieldData("t8413OutBlock1", "high", i)  # 고가
        row[5] = instXAQueryT8413.GetFieldData("t8413OutBlock1", "low", i)  # 저가
        row[6] = instXAQueryT8413.GetFieldData("t8413OutBlock1", "close", i)  # 종가
        row[7] = instXAQueryT8413.GetFieldData("t8413OutBlock1", "jdiff_vol", i)  # 거래량
        row[8] = instXAQueryT8413.GetFieldData("t8413OutBlock1", "value", i)  # 거래대금
        row[9] = instXAQueryT8413.GetFieldData("t8413OutBlock1", "jongchk", i)  # 수정구분
        row[10] = instXAQueryT8413.GetFieldData("t8413OutBlock1", "rate", i)  # 수정비율
        row[11] = instXAQueryT8413.GetFieldData("t8413OutBlock1", "pricechk", i)  # 수정주가반영항목
        row[12] = instXAQueryT8413.GetFieldData("t8413OutBlock1", "ratevalue", i)  # 수정비율반영 거래대금
        row[13] = instXAQueryT8413.GetFieldData("t8413OutBlock1", "sign", i)  # 종가등락구분
        rows.append(list(row))   # 그냥 row를 입력하면 그 전에 입력되었던 row까지 다 현재 값으로 바뀜
    print(g_count, row)
    
    
    j = j + 1
    #if j % 100 == 0:
    #     print('중간 저장을 수행합니다.')
    #   tmp_df = pd.DataFrame(data = rows, columns= column_dailychart)
    #   dailychart = dailychart.append(tmp_df)
    #  #dailychart.to_csv('dailychart3.csv', index=False)
    #  rows = list()
        

print("모든 종목 불러오기 완료")


tmp_df = pd.DataFrame(data = rows, columns= column_dailychart)
dailychart = dailychart.append(tmp_df)
dailychart = dailychart_original.append(dailychart)

dailychart.to_csv('dailychart3.csv', index=False)
    
#if len(rows) >= 0:
#    tmp_df = pd.DataFrame(data = rows, columns= column_dailychart)
#    dailychart = dailychart.append(tmp_df)
#    dailychart.to_csv('dailychart3.csv', index=False)      

print('모든 종목 주가 데이터 저장 완료')



종목정보 가져오는 중...
종목정보  3060 개 가져오기 완료
2982
2982  / 코드 :  335870  / 종목명 :  IBKS제12호스팩
322 ['335870', 2, '20210330', '2170', '2175', '2155', '2170', '5357', '12', '2097152', '0.00', '0', '0', '3']
2983  / 코드 :  335890  / 종목명 :  비올
326 ['335890', 2, '20210330', '2180', '2180', '2100', '2155', '287199', '612', '2097152', '0.00', '0', '0', '5']
2984  / 코드 :  336060  / 종목명 :  유안타제5호스팩
329 ['336060', 2, '20210330', '2085', '2085', '2070', '2080', '5585', '12', '0', '0.00', '0', '0', '3']
2985  / 코드 :  336570  / 종목명 :  대신밸런스제8호스팩
314 ['336570', 2, '20210330', '2095', '2095', '2080', '2090', '68', '0', '0', '0.00', '0', '0', '2']
2986  / 코드 :  337450  / 종목명 :  SK5호스팩
325 ['337450', 2, '20210330', '2090', '2100', '2065', '2095', '2757', '6', '0', '0.00', '0', '0', '2']
2987  / 코드 :  337930  / 종목명 :  브랜드엑스코퍼레이션
153 ['337930', 2, '20210330', '10650', '10800', '10650', '10650', '73992', '791', '0', '0.00', '0', '0', '5']
2988  / 코드 :  338220  / 종목명 :  뷰노
22 ['338220', 2, '20210330', '25200', '25450',

1779 ['900070', 2, '20210330', '1190', '1205', '1165', '1205', '649161', '768', '2097152', '0.00', '0', '0', '2']
3039  / 코드 :  900080  / 종목명 :  에스앤씨엔진그룹
1779 ['900080', 2, '20210330', '197', '197', '197', '197', '0', '0', '2097792', '0.00', '0', '0', '3']
3040  / 코드 :  900100  / 종목명 :  뉴프라이드
1779 ['900100', 2, '20210330', '717', '717', '717', '717', '0', '0', '2097792', '0.00', '0', '0', '3']
3041  / 코드 :  900110  / 종목명 :  이스트아시아홀딩스
1779 ['900110', 2, '20210330', '202', '204', '200', '201', '1061936', '213', '2097152', '0.00', '0', '0', '5']
3042  / 코드 :  900120  / 종목명 :  씨케이에이치
1779 ['900120', 2, '20210330', '334', '334', '323', '330', '641936', '210', '2097152', '0.00', '0', '0', '2']
3043  / 코드 :  900250  / 종목명 :  크리스탈신소재
1268 ['900250', 2, '20210330', '1775', '1815', '1775', '1805', '956963', '1725', '0', '0.00', '0', '0', '2']
3044  / 코드 :  900260  / 종목명 :  로스웰
1166 ['900260', 2, '20210330', '316', '322', '313', '320', '740308', '235', '2097152', '0.00', '0', '0', '2']
3045  / 코드

In [10]:
# 코스피, 코스닥 지수 가져오기


k = 1
j = 0
count = 0
tr_frequency = 1.1

column_dailychart = ['hcode', 'name', 'date', 'open', 'high', 'low', 'close','jdiff_vol', 'value']
row = list(range(len(column_dailychart)))
rows = list()

list_shcode = {'001':'KOSPI', '301':'KOSDAQ'} # 001: 코스피 / 301 : 코스닥
instXAQueryT8419 = win32com.client.DispatchWithEvents("XA_DataSet.XAQuery", XAQueryEventsT8419)
instXAQueryT8419.ResFileName = "C:/eBEST/xingAPI/Res/t8419.res"
instXAQueryT8419.SetFieldData("t8419InBlock", "gubun", 0, "2")  # 2:일, 3:주, 4:월
instXAQueryT8419.SetFieldData("t8419InBlock", "sdate", 0, "20140101") #시작일 (비압축은 최대 500일, 압축은 최대 2000일 까지 나옴)
instXAQueryT8419.SetFieldData("t8419InBlock", "edate", 0, "20210322")  #마지막일
instXAQueryT8419.SetFieldData("t8419InBlock", "comp_yn", 0, 'Y')
isCompressed = True

query_limit =  instXAQueryT8419.GetTRCountLimit('t8419')
query_limit_flag = True
dailychart = pd.DataFrame(data = rows, columns= column_dailychart)

#코스피 불러오기

for shcode in list_shcode:
    print(shcode)
    instXAQueryT8419.SetFieldData("t8419InBlock", "shcode", 0, shcode) 
    instXAQueryT8419.Request(0)

    while XAQueryEventsT8419.query_state == 0:
        pythoncom.PumpWaitingMessages()

    XAQueryEventsT8419.query_state = 0  # 쿼리가 완료되면 다시 상태를 되돌려 놓음

    if isCompressed:
        nDecompSize = instXAQueryT8419.Decompress("t8419OutBlock1")
        pythoncom.PumpWaitingMessages()
        if nDecompSize > 0:
            g_count =  int(instXAQueryT8419.GetFieldData("t8419OutBlock", 'rec_count',0))
        else:
            g_count = 0
    else:
        g_count =  int(instXAQueryT8419.GetFieldData("t8419OutBlock", 'rec_count',0))

    for i in range(g_count):
        row[0] =  shcode
        row[1] = list_shcode[shcode]  # 코스피, 코스닥 여부
        row[2] = instXAQueryT8419.GetFieldData("t8419OutBlock1", "date", i)  # 날짜
        row[3] = instXAQueryT8419.GetFieldData("t8419OutBlock1", "open", i)  # 시가
        row[4] = instXAQueryT8419.GetFieldData("t8419OutBlock1", "high", i)  # 고가
        row[5] = instXAQueryT8419.GetFieldData("t8419OutBlock1", "low", i)  # 저가
        row[6] = instXAQueryT8419.GetFieldData("t8419OutBlock1", "close", i)  # 종가
        row[7] = instXAQueryT8419.GetFieldData("t8419OutBlock1", "jdiff_vol", i)  # 거래량
        row[8] = instXAQueryT8419.GetFieldData("t8419OutBlock1", "value", i)  # 거래대금
        rows.append(list(row))   # 그냥 row를 입력하면 그 전에 입력되었던 row까지 다 현재 값으로 바뀜
    print(g_count, row)
    time.sleep(tr_frequency)
    
print("모든 지수 불러오기 완료")

if len(rows) >= 0:
    tmp_df = pd.DataFrame(data = rows, columns= column_dailychart)

    dailychart = dailychart.append(tmp_df)
    dailychart.to_csv('dailychart_index.csv', index=False)      

print('모든 지수 데이터 저장 완료')



001
1773 ['001', 'KOSPI', '20210322', '3040.01', '3048.11', '3019.60', '3035.46', '915739', '13348889']
301
1773 ['301', 'KOSDAQ', '20210322', '953.04', '957.52', '948.77', '955.38', '1777228', '10545520']
모든 지수 불러오기 완료
모든 지수 데이터 저장 완료


In [10]:
dailychart.loc[dailychart['shcode'] == '950220']   #900260, 900310, 950140, 950180, 950220

,shcode,gubun,date,open,high,low,close,jdiff_vol,value,jongchk,rate,pricechk,ratevalue,sign
4640,950220,2,20160630,3915,4100,3350,3415,62457609,231770,0,0.00,0,0,5
4641,950220,2,20160701,3440,3515,3220,3380,11291832,38040,0,0.00,0,0,5
4642,950220,2,20160704,3445,3485,3360,3375,6041574,20635,0,0.00,0,0,5
4643,950220,2,20160705,3465,3615,3390,3550,8791466,30771,0,0.00,0,0,2
4644,950220,2,20160706,3495,3755,3485,3700,9213471,33269,0,0.00,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5795,950220,2,20210316,339,340,333,336,1373402,462,2097152,0.00,0,0,5
5796,950220,2,20210317,336,341,334,340,1207303,408,2097152,0.00,0,0,2
5797,950220,2,20210318,340,345,337,338,964752,328,2097152,0.00,0,0,5
5798,950220,2,20210319,340,343,335,339,919536,312,2097152,0.00,0,0,2


In [25]:
# 종목 기본정보
def LoadStockItem():
    
    stockitems = pd.DataFrame(columns=['shcode','hname','gubun','etfgubun'])
    i = 0
    rows = list()
    instXAQueryT8430 = win32com.client.DispatchWithEvents("XA_DataSet.XAQuery", XAQueryEvent)
    instXAQueryT8430.ResFileName = "C:/eBEST/xingAPI/Res/t8430.res"
    instXAQueryT8430.SetFieldData("t8430InBlock", "gubun", 0, 0)
    isDone = instXAQueryT8430.Request(0)

    while XAQueryEvent.query_state == 0:
        if(isDone < 0):
            instXAQueryT8430.Request(0)
        pythoncom.PumpWaitingMessages()

    XAQueryEvent.query_state = 0
    count = instXAQueryT8430.GetBlockCount("t8430OutBlock")

    for i in range(count):
        shcode = instXAQueryT8430.GetFieldData("t8430OutBlock","shcode", i) # 종목코드
        hname = instXAQueryT8430.GetFieldData("t8430OutBlock","hname", i) # 종목명
        gubun = instXAQueryT8430.GetFieldData("t8430OutBlock", "gubun", i)  # 코스피, 코스닥 구분
        isETF = instXAQueryT8430.GetFieldData("t8430OutBlock", "etfgubun", i)  # ETF 여부
        #expcode = instXAQueryT8430.GetFieldData("t8430OutBlock", "expcode", i)  # 확장코드
        row = [shcode, hname, gubun, isETF]
        rows.append(row)
    
    stockitems = pd.DataFrame(data= rows, columns=['shcode','hname','gubun','etfgubun'])
    print(stockitems)
    return stockitems


print("종목정보 가져오는 중...")
stockitems = LoadStockItem()   #종목정보 가져오기
stockitems.to_csv('stockitems.csv', index=False)

print("종목정보 가져오기 완료")




종목정보 가져오는 중...
      shcode         hname gubun etfgubun
0     000020          동화약품     1        0
1     000040         KR모터스     1        0
2     000050            경방     1        0
3     000060         메리츠화재     1        0
4     000070         삼양홀딩스     1        0
...      ...           ...   ...      ...
3055  950170           JTC     2        0
3056  950180           SNK     2        0
3057  950190           미투젠     2        0
3058  950200    소마젠(Reg.S)     2        0
3059  950220  네오이뮨텍(Reg.S)     2        0

[3060 rows x 4 columns]
종목정보 가져오기 완료


In [5]:
import pandas as pd
dailychart1 = pd.read_csv('dailychart.csv', dtype={'shcode':str})
dailychart2 = pd.read_csv('dailychart2.csv', dtype={'shcode':str})
dailychart3 = pd.read_csv('dailychart3.csv', dtype={'shcode':str})
dailychart4 = dailychart1.append(dailychart2)
dailychart4 = dailychart4.append(dailychart3)
print(dailychart3['shcode'].unique())

['347000' '347140' '347740' '347770' '347860' '347890' '348030' '348150'
 '348210' '348350' '349720' '351320' '351340' '352700' '352770' '352940'
 '353060' '353070' '353190' '353490' '353810' '354200' '355150' '356860'
 '356890' '357550' '357780' '359090' '363260' '365590' '366330' '367340'
 '367360' '367460' '368770' '369370' '372290' '373200' '900070' '900080'
 '900100' '900110' '900120' '900250' '900260' '900270' '900280' '900290'
 '900300' '900310' '900340' '950110' '950130' '950140' '950160' '950170'
 '950180' '950190' '950200' '950220']


In [7]:
print(len(dailychart4['shcode'].unique()))
print(len(dailychart4))
dailychart4.

3060
4456804


In [5]:
stockitems = pd.read_csv('stockitmes.csv')
print(stockitems)

      shcode         hname  gubun  etfgubun
0     000020          동화약품      1         0
1     000040         KR모터스      1         0
2     000050            경방      1         0
3     000060         메리츠화재      1         0
4     000070         삼양홀딩스      1         0
...      ...           ...    ...       ...
3055  950170           JTC      2         0
3056  950180           SNK      2         0
3057  950190           미투젠      2         0
3058  950200    소마젠(Reg.S)      2         0
3059  950220  네오이뮨텍(Reg.S)      2         0

[3060 rows x 4 columns]


In [11]:
stockitems.loc[stockitems['etfgubun'] == 2]

,shcode,hname,gubun,etfgubun
1396,500001,신한 K200 USD 선물,1,2
1397,500002,신한 USD K200 선물,1,2
1398,500003,신한 인버스 WTI원유,1,2
1399,500004,신한 브렌트원유 선물,1,2
1400,500005,신한 인버스 브렌트원,1,2
...,...,...,...,...
1562,590013,미래에셋 미국 리츠 E,1,2
1563,590018,미래에셋 중국 심천 1,1,2
1564,700001,하나 코스피 변동성추,1,2
1565,700002,하나 KRX300 ETN,1,2


In [10]:
stockitems['etfgubun'].unique()

array([0, 1, 2], dtype=int64)